In [1]:
import nltk
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np

In [2]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-12-10 21:19:40,083 loading file /home/mtnshh/.flair/models/sentiment-en-mix-distillbert_4.pt


In [3]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/mtnshh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
sid.polarity_scores("In case you missed our previous email, can you please take a look at it?")

{'neg': 0.133, 'neu': 0.727, 'pos': 0.139, 'compound': 0.0258}

In [5]:
from textblob import TextBlob

In [6]:
df = pd.read_csv('./gold_dataset.csv')

In [7]:
df['scores'] = df['tweet'].apply(lambda review: sid.polarity_scores(review))

In [8]:
df['compound']  = df['scores'].apply(lambda score_dict: score_dict['compound'])

In [9]:
df['comp_score'] = df['compound'].apply(lambda c: 'POSITIVE' if c >=0 else 'NEGATIVE')

In [10]:
df['scores_text_blob'] = df['tweet'].apply(lambda review: TextBlob(review).sentiment.polarity)
df['comp_score_text_blob'] = df['scores_text_blob'].apply(lambda c: 'POSITIVE' if c >=0 else 'NEGATIVE')

In [11]:
def scores_flair(text):
    s = flair.data.Sentence(text)
    flair_sentiment.predict(s)
    total_sentiment = s.labels
    return str(total_sentiment[0]).split(' ')[0]

df['scores_flair'] = df['tweet'].apply(scores_flair)

In [12]:
s = flair.data.Sentence(['This is bad.', 'Hello'])
flair_sentiment.predict(s)
total_sentiment = s.labels
print(total_sentiment)
print(str(total_sentiment[0]).split(' ')[0]=='POSITIVE')
type(total_sentiment[0])

[NEGATIVE (0.9999)]
False


flair.data.Label

In [13]:
df.head(5)

,id,created_at,tweet,sum,true_value,polarity_api_output,is_same,concept_api_output,conversation_id,date,...,retweet_date,translate,trans_src,trans_dest,scores,compound,comp_score,scores_text_blob,comp_score_text_blob,scores_flair
0,1369812663623180288,2021-03-11 6:19:23,In great news my senior parents are getting th...,2.0,POSITIVE,POSITIVE,1,"['great news', 'senior', 'vaccine', 'thank', '...",1369812663623180288,2021-03-11,...,NaN,NaN,NaN,NaN,"{'neg': 0.0, 'neu': 0.575, 'pos': 0.425, 'comp...",0.8625,POSITIVE,0.800000,POSITIVE,POSITIVE
1,1354514776534220804,2021-01-28 1:11:03,Our Co-Founder and CEO <USER> recently receive...,2.0,POSITIVE,NEGATIVE,0,"['founder', 'recent', 'covid', 'vaccine', 'vac...",1354514776534220804,2021-01-28,...,NaN,NaN,NaN,NaN,"{'neg': 0.0, 'neu': 0.725, 'pos': 0.275, 'comp...",0.9230,POSITIVE,0.516667,POSITIVE,POSITIVE
2,1349045265945264130,2021-01-12 22:57:10,"""This is all we have left. This can't fail."" <...",0.2,POSITIVE,POSITIVE,1,"['fail', 'make', 'vaccine', 'priority', 'plus'...",1349045265945264130,2021-01-12,...,NaN,NaN,NaN,NaN,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,POSITIVE,-0.250000,NEGATIVE,NEGATIVE
3,1366793773728980992,2021-03-02 22:23:24,Dear #MAGA Trump NEVER cared about you. He Jim...,-1.2,NEGATIVE,POSITIVE,0,"['cared', 'stop', 'punch', 'plane', 'traitor',...",1366793773728980992,2021-03-02,...,NaN,NaN,NaN,NaN,"{'neg': 0.183, 'neu': 0.719, 'pos': 0.098, 'co...",-0.3013,NEGATIVE,0.000000,POSITIVE,NEGATIVE
4,1371004616897269763,2021-03-14 13:15:47,I am all for risk triaging and in turn getting...,-1.1,NEGATIVE,POSITIVE,0,"['health care', 'risk', 'turn', 'vaccine', 'va...",1371004616897269763,2021-03-14,...,NaN,NaN,NaN,NaN,"{'neg': 0.042, 'neu': 0.841, 'pos': 0.117, 'co...",0.5789,POSITIVE,0.291667,POSITIVE,NEGATIVE


In [14]:
y_true = df['true_value'].to_numpy()
y_pred_vader = df['comp_score'].to_numpy()
y_pred_sentic = df['polarity_api_output'].to_numpy()
y_pred_text_blob = df['comp_score_text_blob'].to_numpy()
y_pred_flair = df['scores_flair'].to_numpy()

In [15]:
def calc_label(y):
    x = y.copy()
    x[x=="POSITIVE"] = int(1)
    x[x=="NEGATIVE"] = int(-1)
    return x.astype('int64')

In [16]:
def calc_ensemble(y1, y2, y3):
    x1, x2, x3 = calc_label(y1), calc_label(y2), calc_label(y3)
    ans = np.zeros(y1.shape)
    
    ans[(x1+x2+x3)>=0] = 1
    ans[(x1+x2+x3)<0] = -1
    print(x1[:10])
    print(x2[:10])
    print(x3[:10])
    print(ans[:10])
    print(calc_label(y_true)[:10])
    return ans.astype('int64')

In [17]:
print(classification_report(calc_label(y_true), calc_label(y_pred_sentic)))

              precision    recall  f1-score   support

          -1       0.52      0.51      0.52       171
           1       0.75      0.76      0.75       329

    accuracy                           0.67       500
   macro avg       0.64      0.63      0.64       500
weighted avg       0.67      0.67      0.67       500



In [18]:
print(classification_report(calc_label(y_true), calc_label(y_pred_vader)))

              precision    recall  f1-score   support

          -1       0.85      0.51      0.64       171
           1       0.79      0.95      0.86       329

    accuracy                           0.80       500
   macro avg       0.82      0.73      0.75       500
weighted avg       0.81      0.80      0.79       500



In [19]:
print(classification_report(calc_label(y_true), calc_label(y_pred_text_blob)))

              precision    recall  f1-score   support

          -1       0.65      0.37      0.47       171
           1       0.73      0.89      0.81       329

    accuracy                           0.72       500
   macro avg       0.69      0.63      0.64       500
weighted avg       0.70      0.72      0.69       500



In [20]:
print(classification_report(calc_label(y_true), calc_ensemble(y_pred_sentic, y_pred_vader, y_pred_text_blob)))

[ 1 -1  1  1  1  1  1 -1  1  1]
[ 1  1  1 -1  1  1  1  1  1 -1]
[ 1  1 -1  1  1  1  1  1  1  1]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 1  1  1 -1 -1  1 -1  1  1 -1]
              precision    recall  f1-score   support

          -1       0.84      0.42      0.56       171
           1       0.76      0.96      0.85       329

    accuracy                           0.77       500
   macro avg       0.80      0.69      0.70       500
weighted avg       0.79      0.77      0.75       500



In [21]:
print(classification_report(calc_label(y_true), calc_label(y_pred_flair)))

              precision    recall  f1-score   support

          -1       0.60      0.85      0.70       171
           1       0.90      0.70      0.79       329

    accuracy                           0.75       500
   macro avg       0.75      0.78      0.75       500
weighted avg       0.80      0.75      0.76       500

